# Introduction to the Integrated Deepfake Detection System

## Overview

Welcome to the Integrated Deepfake Detection System! This project is a comprehensive effort to tackle the growing problem of deepfake videos using advanced machine learning techniques. As deepfake technology continues to evolve, it poses significant challenges to privacy, security, and authenticity in media. Our system is designed to detect deepfakes by analyzing multiple aspects of video content, specifically focusing on spatial, temporal, and micro-expression features. By integrating these different feature types, we aim to create a robust and reliable detection mechanism capable of identifying manipulated video content.

## Objectives

The primary objective of this project is to develop a deepfake detection system that can accurately distinguish between genuine and manipulated videos. This involves:

1. **Spatial Feature Extraction**: Analyzing individual frames of a video to capture static facial features. This is accomplished using pre-trained Convolutional Neural Networks (CNNs) such as ResNet50 and VGG16, which are fine-tuned for this task.
   
2. **Temporal Feature Extraction**: Understanding how facial features change over time by analyzing sequences of frames. Bidirectional Long Short-Term Memory (BiLSTM) networks are employed to capture temporal dependencies and detect inconsistencies in the flow of facial expressions.

3. **Micro-Expression Analysis**: Focusing on subtle facial movements that are difficult to replicate in deepfake videos. This module uses specialized CNN architectures to extract and analyze micro-expressions, providing an additional layer of detection.

4. **Feature Fusion**: Combining spatial, temporal, and micro-expression features using attention mechanisms to form a comprehensive feature set that enhances detection accuracy. This fusion approach leverages the strengths of each feature type to make a final decision about the authenticity of the video.

## Why This Approach?

Deepfake detection is a challenging task due to the sophistication of the algorithms used to create these fakes. Traditional detection methods that rely on a single type of feature often fail to capture the complexity of manipulations. By integrating spatial, temporal, and micro-expression features, our approach provides a multi-dimensional analysis that significantly improves the likelihood of detecting deepfakes. This holistic strategy addresses the limitations of existing methods and provides a more reliable solution.

### Key Challenges Addressed:

- **Variability in Deepfake Techniques**: Different deepfake algorithms have varying strengths and weaknesses. By analyzing multiple aspects of the video, our system can detect a wide range of manipulations.
- **Subtle Manipulations**: Some deepfakes are so well-crafted that the manipulations are not immediately noticeable to the human eye. Micro-expression analysis helps in detecting these subtle manipulations.
- **Generalization**: Ensuring that the model is not overfitting to specific datasets or types of deepfakes. The fusion of different feature types helps the model generalize better across diverse datasets.

## Dataset

For this project, we use the **FaceForensics++** dataset, which is a benchmark dataset commonly used in deepfake detection research. It contains a collection of both original and manipulated video sequences, providing a diverse set of examples for training and testing the system. The dataset is divided into two main categories:

- **Original Sequences**: Videos that have not been altered, serving as ground truth for authenticity.
- **Manipulated Sequences**: Videos that have been altered using various deepfake techniques, providing examples of fake content.

## Notebook Structure

This Jupyter Notebook is structured to guide you through the different stages of the deepfake detection process:

1. **Data Preprocessing**: Preparing the video frames for feature extraction, including face detection, alignment, and normalization.
   
2. **Model Architecture**: Detailed implementation of the spatial, temporal, and micro-expression feature extraction models. This section includes building and training the CNN (ResNet50, VGG16), BiLSTM, and micro-expression analysis models.

3. **Feature Fusion and Classification**: Combining the extracted features and using attention mechanisms to improve the detection accuracy. The final output layer provides the classification result, indicating whether the video is genuine or a deepfake.

4. **Evaluation and Results**: Testing the trained model on a set of test videos and evaluating its performance using metrics such as accuracy, precision, recall, and F1-score.

5. **Conclusion and Future Work**: Summarizing the findings and discussing potential improvements and future directions for research in deepfake detection.

## Getting Started

To run this notebook, ensure that you have all the necessary dependencies installed. The required libraries are listed in the `requirements.txt` file. Use the following command to install them:

```bash
pip install -r requirements.txt
```

Additionally, make sure you have the FaceForensics++ dataset downloaded and placed in the appropriate directory as specified in the notebook.

## Conclusion

The Integrated Deepfake Detection System represents a significant step towards addressing the challenges posed by deepfake technology. By leveraging multiple feature extraction methods and incorporating attention mechanisms, this system aims to provide a robust solution for identifying manipulated video content. We hope that this project will contribute to the ongoing efforts to maintain the integrity and authenticity of digital media.

Let's get started and dive into the world of deepfake detection!

## **Data Loading**

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
original_video_directory = '../Datasets/original_sequences'
manipulated_video_directory = '../Datasets/manipulated_sequences'

In [3]:
video_paths = []
labels = []

for  video in os.listdir(original_video_directory):
    if video.endswith('.mp4'):
        video_paths.append(os.path.join(original_video_directory, video))
        labels.append(0)
for video in os.listdir(manipulated_video_directory):
    if video.endswith('.mp4'):
        video_paths.append(os.path.join(manipulated_video_directory,video))
        labels.append(1)

deepfake_data = pd.DataFrame({
    'video_path': video_paths,
    'label': labels
})

deepfake_data = deepfake_data.sample(frac=1).reset_index(drop=True)

In [4]:
deepfake_data['label'].value_counts()

label
1    56
0    36
Name: count, dtype: int64

In [5]:
deepfake_data.head(5)

,video_path,label
0,../Datasets/manipulated_sequences\02_01__outsi...,1
1,../Datasets/manipulated_sequences\01_15__exit_...,1
2,../Datasets/original_sequences\12__podium_spee...,0
3,../Datasets/original_sequences\05__kitchen_sti...,0
4,../Datasets/original_sequences\08__outside_tal...,0


## **Preprocessing Phase**

## **Model Building**

## 1. **Facial Feature Extractor Module**

### 1. Spatial Feature Extractor

In [6]:
import numpy as np
import cv2
import os
import tensorflow as tf
from keras.api.layers import TimeDistributed,Input, Flatten
from keras.api.applications.resnet50 import  ResNet50, preprocess_input as resnet_preprocess
from keras.api.models import  Model
import matplotlib.pyplot as plt

In [7]:
base_model = ResNet50(include_top=False, weights='imagenet', pooling='avg', input_shape=(224,224,3))

In [8]:
spatial_model =  Model(inputs=base_model.input, outputs=base_model.output)

In [9]:
def resnet_preprocess(image_path, target_size=(224,224)):

    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, target_size)

    img = img/255.0
    img = (img-0.5)*2.0

    img_arr = tf.expand_dims(img, axis=0)

    return img_arr


In [10]:
preprocessable_files = []
for filename in os.listdir('../frames'):
    filepath = os.path.join('../frames',filename)
    preprocessable_files.append(filepath)

In [11]:
preprocessed_images = []
for item in preprocessable_files:
    preprocessed_img = resnet_preprocess(item)
    preprocessed_images.append(preprocessed_img)
preprocessed_images_batch = tf.concat(preprocessed_images, axis=0)

In [12]:
len(preprocessed_images_batch)

146

In [13]:
preprocessed_images = np.array(preprocessed_images_batch)

In [14]:
features = spatial_model.predict(preprocessed_images)

5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step


In [15]:
preprocessed_images.shape

(146, 224, 224, 3)

In [16]:
preprocessed_images_expanded = np.expand_dims(preprocessed_images, axis=0)

In [17]:
preprocessed_images_expanded.shape

(1, 146, 224, 224, 3)

In [18]:
features.shape

(146, 2048)

In [19]:
print(features[0])

[0.         0.         0.00772802 ... 4.02363    0.         0.        ]


### 2. Temporal Feature Extractor

In [20]:
from tensorflow import keras
from keras.layers import Bidirectional,LSTM,Input
from keras.models import Model

In [21]:
def build_temporal_feature_extractor(sequence_length, feature_shape):
    input_seq = Input((sequence_length, feature_shape))

    lstm_1 = LSTM(128, return_sequences=True, dropout=0.2)
    lstm_2 = LSTM(64,return_sequences=True, dropout=0.2)

    lstm_out = Bidirectional(lstm_1)(input_seq)
    lstm_out = Bidirectional(lstm_2)(lstm_out)

    model = Model(inputs=input_seq, outputs=lstm_out)
    return model

In [22]:
temporal_feature_extractor = build_temporal_feature_extractor(sequence_length=features.shape[0], feature_shape=features.shape[1])

In [23]:
temporal_feature_extractor.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 146, 2048)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 146, 256)       │     2,229,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 146, 128)       │       164,352 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,393,600 (9.13 MB)

 Trainable params: 2,393,600 (9.13 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
features.shape

(146, 2048)

In [25]:
features_temp = np.expand_dims(features, axis=0)

In [26]:
features_temp.shape

(1, 146, 2048)

In [27]:
temp_features_extracted = temporal_feature_extractor.predict(features_temp)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 561ms/step


In [28]:
temp_features_extracted

array([[[-0.06738199,  0.08338698, -0.00320563, ...,  0.03455913,
         -0.08465538, -0.48169863],
        [-0.12192948,  0.17146903,  0.00371655, ...,  0.02755469,
         -0.09819572, -0.47330675],
        [-0.16106763,  0.24418211,  0.01061229, ...,  0.02497226,
         -0.10267814, -0.46580225],
        ...,
        [-0.26383302,  0.41573468,  0.0013749 , ...,  0.0721532 ,
         -0.0936614 , -0.04470664],
        [-0.2691552 ,  0.4132893 ,  0.0051184 , ...,  0.05087633,
         -0.08179858, -0.01124781],
        [-0.27371392,  0.3983916 ,  0.01492638, ...,  0.02353151,
         -0.05558356,  0.00660072]]], dtype=float32)

## **2. Micro Expression Inconsistency Detection Module**

### **1. Micro Expression Feature Extraction**

In [29]:
from keras.api.layers import Conv2D,BatchNormalization,Activation,MaxPooling2D,Dropout, Dense, Flatten

In [30]:
def build_micro_exp_spatial_feature_extractor(spatial_input_shape):
    spatial_inputs = Input(spatial_input_shape)

    # Layer 1
    micro_exp_x = Conv2D(32,(3,3),padding='same')(spatial_inputs)
    micro_exp_x = BatchNormalization()(micro_exp_x)
    micro_exp_x = Activation('relu')(micro_exp_x)
    micro_exp_x = MaxPooling2D(pool_size=(2,2))(micro_exp_x)
    
    # Layer 2
    micro_exp_x = Conv2D(64,(3,3),padding='same')(micro_exp_x)
    micro_exp_x = BatchNormalization()(micro_exp_x)
    micro_exp_x = Activation('relu')(micro_exp_x)
    micro_exp_x = MaxPooling2D(pool_size=(2,2))(micro_exp_x)
    
    # Layer 3
    micro_exp_x = Conv2D(128,(3,3),padding='same')(micro_exp_x)
    micro_exp_x = BatchNormalization()(micro_exp_x)
    micro_exp_x = Activation('relu')(micro_exp_x)
    micro_exp_x = MaxPooling2D(pool_size=(2,2))(micro_exp_x)
    
    # Layer 4
    micro_exp_x = Flatten()(micro_exp_x)
    micro_exp_x = Dense(256, activation='relu')(micro_exp_x)
    # Add dropouts in case of overfitting
    
    micro_exp_output = Dense(128, activation='relu')(micro_exp_x)
    
    micro_exp_spatial_feature_extractor = Model(inputs=spatial_inputs, outputs=micro_exp_output)
    
    return micro_exp_spatial_feature_extractor

In [31]:
# input_shape = (64,64,3)
# inputs = Input(input_shape)
# 
# # Layer 1
# x = Conv2D(32,(3,3),padding='same')(inputs)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)
# 
# # Layer 2
# x = Conv2D(64,(3,3),padding='same')(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)
# 
# # Layer 3
# x = Conv2D(128,(3,3),padding='same')(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)
# 
# # Layer 4
# x = Flatten()(x)
# x = Dense(256, activation='relu')(x)
# # Add dropouts in case of overfitting
# 
# output = Dense(128, activation='relu')(x)
# 
# micro_exp_feature_extractor = Model(inputs=inputs, outputs=output)
# micro_exp_feature_extractor.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# micro_exp_feature_extractor.summary()

In [32]:
# #TODO: Train the model
# micro_exp_feature_extractor.fit()

### **2. Temporal Inconsistency Detection in Micro Expression**

In [33]:
from keras.api.layers import Attention,Concatenate

In [34]:
def build_temporal_inconsistency_attention(seq_length, feature_dims):
    temp_feat_shape = (seq_length, feature_dims)
    temp_inputs = Input(shape=temp_feat_shape)
    
    x_mic_exp = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2))(temp_inputs)
    
    x_mic_exp = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2))(x_mic_exp)
    
    attention_output = Attention()([x_mic_exp,x_mic_exp])
    
    x_mic_exp = Dense(256, activation='relu')(attention_output)
    # Add dropout layers for overfitting
    x_mic_exp = Dense(128, activation='relu')(x_mic_exp)
    
    mic_exp_temp_model = Model(inputs=temp_inputs, outputs=x_mic_exp)
    
    mic_exp_temp_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    
    return mic_exp_temp_model
    

## **3. Feature Fusion Layer**

### **1. Spatial Attention Mechanism**

In [35]:
from keras.api.layers import Multiply, GlobalAvgPool2D

In [36]:
def build_spatial_attention_mechanism(feature_maps):
    """
    :param feature_maps: 
    :return: weighted feature maps
    """
    
    attention_map = Conv2D(1,(1,1),strides=(1,1),padding="same")(feature_maps)
    
    attention_map = Activation('sigmoid')(attention_map) # 'sigmoid' or 'softmax' can be used as an activation function
    
    # Element wise multiplication of feature_maps and attention_map
    weighted_feature_map = Multiply()([feature_maps, attention_map])
    
    # Convert the weighted feature map into a context vector
    spatial_context_vectors = GlobalAvgPool2D()(weighted_feature_map)
    
    return spatial_context_vectors
    

### **2. Temporal Attention Mechanism**

In [37]:
def build_temporal_attention_mechanism(feature_maps):
    """
    :param feature_maps: 
    :return weighted_feature_maps: 
    """
    
    temporal_attention_scores = Dense(1, activation='tanh')(feature_maps)
    
    temporal_attention_weights = Activation('softmax')(temporal_attention_scores)
    
    weighted_temporal_features = Multiply()([feature_maps, temporal_attention_weights])
    
    context_vector = tf.reduce_sum(weighted_temporal_features, axis=1)
    
    return context_vector

### **3. Micro Expression Attention Mechanism**

#### **1. Spatial Micro Expression Attention Mechanism**

In [38]:
def build_spatial_micro_expression_attention_mechanism(micro_exp_spatial_feature_maps):
    """
    :param micro_exp_spatial_feature_maps: 
    :return weighted micro_exp feature maps : 
    """
    
    attention_map = Conv2D(1,(1,1),strides=(1,1),padding="same")(micro_exp_spatial_feature_maps)
    
    attention_map = Activation('sigmoid')(attention_map)
    
    weighted_micro_exp_feature_map = Multiply()([micro_exp_spatial_feature_maps,attention_map])
    
    micro_exp_spatial_context_vector = GlobalAvgPool2D()(weighted_micro_exp_feature_map)
    
    return micro_exp_spatial_context_vector

#### **2. Temporal Micro Expression Attention Mechanism**

In [39]:
def build_temporal_micro_expression_attention_mechanism(micro_exp_feature_vectors):
    """
    :param micro_exp_feature_vectors: 
    :return micro_exp_context_vectors: 
    """
    
    attention_scores = Dense(1,activation='tanh')(micro_exp_feature_vectors)
    
    attention_weights = Activation('softmax')(attention_scores)
    
    weighted_micro_exp_temporal_features = Multiply()([attention_weights, micro_exp_feature_vectors])
    
    micro_exp_context_vector = tf.reduce_sum(weighted_micro_exp_temporal_features, axis=1)
    
    return micro_exp_context_vector

### **4. Concatenation Layer**

In [40]:
from keras.api.layers import Concatenate

In [41]:
def build_feature_fusion_layer(spatial_features, temporal_features, micro_exp_spatial_features, micro_exp_temporal_features):
    
    spatial_context_vectors = build_spatial_attention_mechanism(feature_maps=spatial_features)

    temporal_context_vector = build_temporal_attention_mechanism(feature_maps=temporal_features)
    
    micro_exp_spatial_context_vector = build_spatial_micro_expression_attention_mechanism(micro_exp_spatial_feature_maps=micro_exp_spatial_features)
    
    micro_exp_temporal_context_vector = build_temporal_micro_expression_attention_mechanism(micro_exp_feature_vectors=micro_exp_temporal_features)
    
    concatenated_feature_vector = Concatenate()([
        spatial_context_vectors,
        temporal_context_vector,
        micro_exp_spatial_context_vector,
        micro_exp_temporal_context_vector
    ])
    
    return concatenated_feature_vector

## **4. Face Swap Detection Model**

In [42]:
def build_face_swap_detection_model(concatenated_feature_vector):
    
    dense_units = [256,128,64]
    
    x_face_swap = concatenated_feature_vector
    for unit in dense_units:
        x_face_swap = Dense(unit, activation='relu')(x_face_swap)
        x_face_swap = Dropout(0.5)(x_face_swap)
    
    op_face_swap = Dense(1,activation='sigmoid')(x_face_swap)
    
    face_swap_detector_model = Model(inputs=concatenated_feature_vector, output=op_face_swap)
    
    return face_swap_detector_model    

## **Face Swap Detection Pipeline**

In [43]:
def build_face_swap_detection_pipeline(spatial_feature_dims, temporal_feature_dims, micro_exp_spatial_feature_dims, micro_exp_temporal_feature_dims, temporal_seq_length, micro_exp_temp_seq_length):

    #define models for feature extractor modules
    spatial_feature_input =  Input(shape=(spatial_feature_dims,))
    temporal_feature_input = Input(shape=(temporal_feature_dims,))
    micro_exp_spatial_feature_input = Input(shape=(micro_exp_spatial_feature_dims,))
    micro_exp_temporal_feature_input = Input(shape=(micro_exp_temporal_feature_dims,))
    
    # build feature extractor modules
    spatial_feature_extractor = spatial_model
    temporal_feature_extractor_ = build_temporal_feature_extractor(temporal_seq_length,(temporal_feature_dims,))
    micro_exp_spatial_feature_extractor = build_micro_exp_spatial_feature_extractor(micro_exp_spatial_feature_input)
    micro_exp_temporal_inconsistency_detector = build_temporal_inconsistency_attention(micro_exp_temp_seq_length,micro_exp_temporal_feature_input)
    
    #extract features
    spatial_features = spatial_feature_extractor()
    micro_exp_spatial_features = micro_exp_spatial_feature_extractor()
    temporal_features = temporal_feature_extractor(spatial_features)
    micro_exp_temporal_features = micro_exp_temporal_inconsistency_detector(micro_exp_spatial_features)
    
    #build the feature fusion layer with the extracted features as an input
    concatenated_feature_vector = build_feature_fusion_layer(spatial_features, temporal_features, micro_exp_spatial_features, micro_exp_temporal_features)
    
    
    #now build the final face detection classification model
    face_swap_detector_model = build_face_swap_detection_model(concatenated_feature_vector)
    
    
    return face_swap_detector_model
    